In [1]:
pip install transformers datasets torch pandas

In [3]:
import pandas as pd
from transformers import AutoConfig
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback
)
from datasets import Dataset

# Load dataset
df = pd.read_csv('/content/MathRiddlesForFineTune.csv')

# Format data
formatted_data = []
for _, row in df.iterrows():
    text = (
        f"Riddle: {row['Riddle']}\n"
        f"Answer: {row['Solution']}\n\n"
    )
    formatted_data.append({"text": text})

# Convert to Dataset
dataset = Dataset.from_list(formatted_data)
train_val_split = dataset.train_test_split(test_size=0.2, seed=42)

# Load model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Correct tokenization (no padding or return_tensors)
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=128  # Truncate long sequences
    )

tokenized_datasets = train_val_split.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

# Data collator for dynamic padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=15,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    logging_steps=15,
    gradient_accumulation_steps=2,
    max_grad_norm=1.0,
    fp16=True,
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train
trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alishbanazir910 (alishbanazir910-fast-national-university-of-computing-an) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,4.641693
2,No log,3.242150
3,No log,2.710022
4,No log,2.354197
5,3.226700,2.068899
6,3.226700,1.872416
7,3.226700,1.739900
8,3.226700,1.635544
9,3.226700,1.563325
10,1.724000,1.515262


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=45, training_loss=2.095218065049913, metrics={'train_runtime': 371.6163, 'train_samples_per_second': 0.969, 'train_steps_per_second': 0.121, 'total_flos': 5119690752000.0, 'train_loss': 2.095218065049913, 'epoch': 15.0})

In [4]:
# Save the model
model.save_pretrained("./math_riddle_generator")

# Save the tokenizer
tokenizer.save_pretrained("./math_riddle_generator")

('./math_riddle_generator/tokenizer_config.json',
 './math_riddle_generator/special_tokens_map.json',
 './math_riddle_generator/vocab.json',
 './math_riddle_generator/merges.txt',
 './math_riddle_generator/added_tokens.json',
 './math_riddle_generator/tokenizer.json')

In [ ]:
from transformers import pipeline
import re


model_path = "./math_riddle_generator"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)


generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    pad_token_id=tokenizer.eos_token_id,
)


prompt = "Riddle: What number becomes zero when you subtract 15 from half of it?\nAnswer:"

output = generator(
 prompt,
    max_length=50,
    do_sample=True,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
    truncation=True,

)

# Extract the answer using regex
generated_text = output[0]["generated_text"]
match = re.search(r"Answer:\s*(\d+)", generated_text)

if match:
    answer = match.group(1)
    print(f"Answer: {answer}")
else:
    print("No valid answer found.")

Device set to use cuda:0


Answer: 30


In [5]:
from transformers import pipeline


model_path = "./math_riddle_generator"
generator = pipeline("text-generation", model=model_path, device=0)  # Ensure CUDA is used

# Define a template for riddle generation
prompt = "Math Riddle:\n<Q> "

# Generate riddles automatically
num_riddles = 5
riddles = []

print("🔹 Generating riddles...")

for _ in range(num_riddles):
    output = generator(
        prompt,
        max_length=50,
        do_sample=True,
        temperature=0.7,
        top_k=40,
        top_p=0.9,
        repetition_penalty=1.2,
        truncation=True
    )

    print(f"🔹 Raw Output: {output}")  # Debug print

    if output:
        generated_text = output[0]['generated_text'].strip()

        if "<A>" in generated_text:
            question, answer = generated_text.split("<A>", 1)
            riddle = f"{question.strip()}\n<A>{answer.splitlines()[0].strip()}"
            riddles.append(riddle)
        else:
            riddles.append(generated_text)  # Store raw text if no <A>

print("🔹 Finished generating riddles!\n")

# Print final riddles
if riddles:
    for i, riddle in enumerate(riddles, 1):
        print(f"Riddle {i}:\n{riddle}\n")
else:
    print("⚠️ No riddles were generated. Check model output.")



Device set to use cuda:0


🔹 Generating riddles...
🔹 Raw Output: [{'generated_text': 'Math Riddle:\n<Q> -------------------------- What is the sum of two numbers?\n\nAnswer to 9 divided by 4 equals 12. This number will be multiplied twice when you multiply it with a decimal place (2). The result should then'}]
🔹 Raw Output: [{'generated_text': "Math Riddle:\n<Q> -------------------------- What number am I?</q>\nAnswer - 20\n\n\xa0What is your answer to the question?\n\nAnswer: 25. You get 30 for 1 and 15 for 2. That's"}]
🔹 Raw Output: [{'generated_text': 'Math Riddle:\n<Q> ------------------------- What number is 7?\n\nAnswer (12): 30. If you subtract 12 from it, your answer becomes 20. Your error will be 40 and the result of multiplying by 10 equals 4.'}]
🔹 Raw Output: [{'generated_text': 'Math Riddle:\n<Q> ---------------------------{1, 2}\n\n. What number is the sum of 10 and 40? Now multiply by 16 to get 32! The result should be 8; add 25 for each digit you divide it'}]
🔹 Raw Output: [{'generated_text': 'Mat

In [6]:
pip install gradio


In [7]:
import gradio as gr
from transformers import pipeline

# Load fine-tuned model
model_path = "./math_riddle_generator"
generator = pipeline("text-generation", model=model_path, device=0)  # Use GPU if available

def generate_riddles():
    """Generates 5 math riddles using the fine-tuned model."""
    prompt = "Math Riddle:\n<Q> "
    num_riddles = 5
    riddles = []

    for _ in range(num_riddles):
        output = generator(
            prompt,
            max_length=50,
            do_sample=True,
            temperature=0.7,
            top_k=40,
            top_p=0.9,
            repetition_penalty=1.2,
            truncation=True
        )

        if output:
            generated_text = output[0]['generated_text'].strip()

            if "<A>" in generated_text:
                question, answer = generated_text.split("<A>", 1)
                riddle = f"{question.strip()}\n<A>{answer.splitlines()[0].strip()}"
                riddles.append(riddle)
            else:
                riddles.append(generated_text)  # Store raw text if no <A>

    return "\n\n".join([f"🔹 Riddle {i+1}:\n{riddle}" for i, riddle in enumerate(riddles)])

# Create Gradio Interface
iface = gr.Interface(
    fn=generate_riddles,
    inputs=None,  # No user input needed, generates riddles automatically
    outputs="text",
    title="🤔 Math Riddle Generator",
    description="Click the **Generate Riddles** button to get 5 unique math riddles!",
    live=False
)

# Launch the Gradio app
iface.launch(share=True)


Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://59351fb795092edd6b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
